# Scratch Artificial Neural Network

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.dont_write_bytecode = True

# Import libraries
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Import custom modules
from ArtificialNeuralNetwork import ArtificialNeuralNetwork
from Layer import Layer, OutputLayer, InputLayer
from Function import ActivationFunction, LossFunction, AdaptiveOptimizer
from enums import InitializerType, RegularizationType

# Pipeline for testing MNIST dataset

In [ ]:
input_size = 784
hidden_layers = 2
hidden_size = 128
output_size = 10
learning_rate = 0.001
param_1 = 0
param_2 = 0
batch_size = 64

In [ ]:
# Load MNIST dataset using fetch_openml
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
X = X / 255.0
y = y.astype(int)


# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42,
)

In [ ]:
ann = ArtificialNeuralNetwork(
        123,
        InputLayer(input_size=input_size),
        Layer(
            weight_init=InitializerType.HE,
            bias_init=InitializerType.ZERO,
            input_size=input_size,
            num_neurons=128,
            param_1=param_1,
            param_2=param_2,
            activation=ActivationFunction.prelu,
            regularizer=RegularizationType.L2,
            optimizer="sgd",
            alpha=0.45,
            gamma=0.95,
            layer_name=f"Hidden Layer 0",
        ),
        *[Layer(
            weight_init=InitializerType.HE,
            bias_init=InitializerType.ZERO,
            input_size=128,
            num_neurons=128,
            param_1=param_1,
            param_2=param_2,
            activation=ActivationFunction.prelu,
            regularizer=RegularizationType.L2,
            optimizer="sgd",
            alpha=0.45,
            gamma=0.95,
            layer_name=f"Hidden Layer {_}",
        ) for _ in range(hidden_layers - 1)],
        OutputLayer(
            weight_init=InitializerType.HE,
            bias_init=InitializerType.ZERO,
            input_size=128,
            num_neurons=output_size,
            param_1=param_1,
            param_2=param_2,
            activation=ActivationFunction.softmax,
            regularizer=RegularizationType.L2,
            optimizer="sgd",
            loss_funct=LossFunction.categorical_cross_entropy,
            layer_name="Output Layer"
        )
    )

In [ ]:
ann.train(
    x=X_train,
    y=y_train,
    loss_function=LossFunction.categorical_cross_entropy,
    lr=learning_rate,
    epochs=50,
    batch_size=batch_size,
    verbose=True,
    validation_data=(X_test, y_test),
    useRMSProp=False
)

In [ ]:
from sklearn.metrics import f1_score

y_pred = ann.predict(X_test)
print(f"F1 Score: {f1_score(y_test, y_pred, average='macro')}")

In [ ]:
ann.visualize_weight_distribution((1,2,3))

In [ ]:
ann.save("ann_model.pkl")

In [ ]:
new_model = ArtificialNeuralNetwork()

In [ ]:
new_model.load("ann_model.pkl")

In [ ]:
new_model.evaluate(X_test, y_test)

In [ ]:
# Compare with MLP

from sklearn.neural_network import MLPClassifier

# Comparison with scikit-learn MLP (using ReLU instead of PReLU)
model = MLPClassifier(
    hidden_layer_sizes=(128, 128),
    activation='relu',
    solver='sgd',
    max_iter=50,
    batch_size=batch_size,
    random_state=123,
    verbose=True,
    learning_rate_init=0.01,
    learning_rate='constant',
    tol=0.0
)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("F1 Score: ", f1_score(y_test, y_pred, average='macro'))